In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os
import json
import time
import concurrent.futures
import pandas as pd


In [ ]:
cookie = 'Bifrost_SessionID=q0svamuliir35pmr0xag1c5t'

In [ ]:
def part_text(part,start,end):
    part_list = []
    for i in part[start:end]:
        part_list.append(i.get_text().replace('\r', '').replace('\n', '').strip())
        part_text = '\n'.join(part_list)
    return part_text
        

In [ ]:
def get_id_details(cookie,doc_id):
    # URL cần gửi yêu cầu
    url = f"https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateDisplay.aspx?DocId={doc_id}"

    # Headers cần thiết
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7",
        "Cache-Control": "max-age=0",
        "Connection": "keep-alive",
        "Cookie": cookie,
        "DNT": "1",
        "Referer": "https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
    }

    # Gửi yêu cầu GET
    response = requests.get(url, headers=headers)

    # Kiểm tra phản hồi
    if response.status_code == 200:
        print("Request thành công!")
        # print(response.text)  # In nội dung HTML của trang
    else:
        print(f"Lỗi: {response.status_code}")
        # print(response.text)

    id_info = {}
    id_info["doc_id"] = doc_id
    if response.text:
        soup = BeautifulSoup(response.text,"html.parser")

    info_soup = soup.find('div',id = "ctl00_cplhContainer_rpvInfo") 
    all_div = info_soup.find_all('div', class_=re.compile(r"^row"))
    trang_thai_soup = all_div[0]
    try :
        tths = part_text(trang_thai_soup.find_all('span'),1,3)
        id_info["Trạng thái hồ sơ"] = tths
    except:
        print("not found Trạng thái hồ sơ")
    thong_tin_chung_soup = all_div[1]
    try:
        tax_code = thong_tin_chung_soup.find("span", id = "ctl00_cplhContainer_lblCompanyTaxCode").get_text().strip()
        id_info["Tax code"] = tax_code
    except:
        print("Not found tax_code")
    try:
        issuing_authority  = thong_tin_chung_soup.find("span", id = "ctl00_cplhContainer_lblDepartment").get_text().strip()
        id_info["Issuing Authority"] = issuing_authority
    except:
        print("Not found Issuing Authority")
    try:
        form  = thong_tin_chung_soup.find("span", id = "ctl00_cplhContainer_lblFormCO").get_text().strip()
        id_info["Form"] = form
    except:
        print("Not found form")
    try:
        ImportingCountry   = thong_tin_chung_soup.find("span", id = "ctl00_cplhContainer_lblMarket").get_text().strip()
        id_info["Importing_Country"] = ImportingCountry
    except:
        print("Not found Importing Country")
    try:
        Reference_no    = thong_tin_chung_soup.find("span", id = "ctl00_cplhContainer_lblCertificateNumber").get_text().strip()
        id_info["Reference No."] = Reference_no
    except:
        print("Not found Reference No.")
    try:
        Issuance_date = thong_tin_chung_soup.find("span", id = "ctl00_cplhContainer_lblCertificateDate").get_text().strip()
        id_info["Issuance date"] = Issuance_date
    except:
        print("Not found Issuance date ")
    try:
        Certified_date  = thong_tin_chung_soup.find("span", id = "ctl00_cplhContainer_lblReviewDateApprove").get_text().strip()
        id_info["Certified date"] = Certified_date
    except:
        print("Not found Certified date ")
        
    try:
        invoiceItems  = thong_tin_chung_soup.find('div',  id = "ctl00_cplhContainer_pnlCustomsNumber").find_all('div',class_ = "invoiceItem")
    except:
        invoiceItems = []
        
    ExportDeclaration = []
    item_num = 0
    for item in invoiceItems:
        invoiceitem_info = {}
        item_num += 1
        invoiceitem_info["item_num"] = item_num
        
        try: 
            invoice_number = item.find("span", id=lambda x: x and x.startswith("ctl00_cplhContainer_pnlCustomsNumber") and "lblInvoiceNumber" in x)
            invoiceitem_info['invoice_number'] = invoice_number.get_text() 
        except:
            invoiceitem_info['invoice_number'] = ""
            
        try:    
            invoice_date = item.find("span", id=lambda x: x and x.startswith("ctl00_cplhContainer_pnlCustomsNumber") and x.endswith("lblInvoiceDate"))
            invoiceitem_info['invoice_date'] = invoice_date.get_text() 
        except:
            invoiceitem_info['invoice_date'] = ""
            
        try: 
            invoice_link = item.find("a", id=lambda x: x and x.startswith("ctl00_cplhContainer_pnlCustomsNumber") and "hplInvoiceLink" in x)
            invoiceitem_info['invoice_link'] = invoice_link.get("href")
        except:
            invoiceitem_info['invoice_link'] = ""
            
        ExportDeclaration.append(invoiceitem_info)

    id_info["ExportDeclaration"] = ExportDeclaration
    from_to_soup = all_div[2]
    titles = from_to_soup.find_all('span', class_='titles')
    for title in titles:
        key = title.get_text(strip=True)
        div_content = title.find_parent('div').find_next_sibling('div')
        values = [span.get_text(strip=True) for span in div_content.find_all('span')]
        id_info[key] = values

    transport_info_soup = all_div[3]
    try:
        transport_type  = transport_info_soup.find("span", id = "ctl00_cplhContainer_lblTransportMethod").get_text().strip()
        id_info["transport_type"] = transport_type
    except:
        print("Not found transport_type")
    try:
        departure_date  = transport_info_soup.find("span", id = "ctl00_cplhContainer_lblTransportDate").get_text().strip()
        id_info["Departure Date"] = departure_date
    except:
        print("Not found Departure Date")
    try:
        vessel_aircraft_name  = transport_info_soup.find("span", id = "ctl00_cplhContainer_lblShipName").get_text().strip()
        id_info["vessel_aircraft_name"] = vessel_aircraft_name
    except:
        print("Not found Vessel’s Name/Aircraft")
    try:
        port_of_loading  = transport_info_soup.find("span", id = "ctl00_cplhContainer_lblSenderPlace").get_text().strip()
        id_info["port_of_loading"] = port_of_loading
    except:
        print("Not found port_of_loading")
    try:
        port_of_discharge  = transport_info_soup.find("span", id = "ctl00_cplhContainer_lblReceiverPlace").get_text().strip()
        id_info["port_of_discharge"] = port_of_discharge
    except:
        print("Not found port_of_discharge")
    try:
        transport_doc  = transport_info_soup.find("a", id = "ctl00_cplhContainer_hplTransportFileLink").get("href")
        id_info["transport_doc"] = transport_doc
    except:
        print("Not found transport_doc")
    try:
        goods_info_soup = all_div[4].find('table',class_ = 'rgMasterTable')
        # Lấy tiêu đề
        goods_info_header = [th.get_text(separator=" ", strip=True) for th in goods_info_soup.find('thead').find_all('th', class_ = "rgHeader")]

        # Lấy dữ liệu từng dòng trong bảng
        goods_info_rows = []
        for tr in goods_info_soup.find('tbody').find_all('tr'):
            row = [td.get_text(separator=" ", strip=True) for td in tr.find_all('td')]
            goods_info_rows.append(row)
    except:
        print("Not found goods_info")
    goods_info = [goods_info_header] + goods_info_rows
    id_info["goods_info"] = goods_info
    invoice_info_soup = all_div[5]
    try:
        total_value  = invoice_info_soup.find("span", id = "ctl00_cplhContainer_lblTotalItemFobValueTabCO").get_text().strip()
        id_info["total_value"] = total_value
    except:
        print("Not found total_value")
    try:
        total_quantity  = invoice_info_soup.find("span", id = "ctl00_cplhContainer_lblTotalItemQuantityTabCO").get_text().strip()
        id_info["total_quantity"] = total_quantity
    except:
        print("Not found total_quantity")
    try:
        total_weight  = invoice_info_soup.find("span", id = "ctl00_cplhContainer_lblTotalItemGrossWeightTabCO").get_text().strip()
        id_info["total_weight"] = total_weight
    except:
        print("Not found total_weight")
    try:
        show_invoi_checkbox = invoice_info_soup.find("input",  id = "ctl00_cplhContainer_ckbShowOnCO")
        checked = show_invoi_checkbox.has_attr("checked")
        id_info["is_show_fob_value"] = checked
    except:
        print("Not found show_invoi_checkbox")
    try:
        origindoc_soup = invoice_info_soup.find("div",id = "ctl00_cplhContainer_pnlFileCoriginalContent").find_all("a")
        origindoc = [a.get("href") for a in origindoc_soup]
        id_info["origindoc"] = origindoc
    except:
        print("Not found origindoc")
    try:
        invoice_attached_soup = invoice_info_soup.find("div",id = "ctl00_cplhContainer_pnlInvoice").find_all("a")
        invoice_attached = [a.get("href") for a in invoice_attached_soup]
        id_info["invoice_attached"] = invoice_attached
    except:
        print("Not found invoice_attached")
    other_info_soup = all_div[6]
    try:
        declaration_place = other_info_soup.find("span",id = "ctl00_cplhContainer_lblCountryCode").get_text().strip()
        id_info["declaration_place"] = declaration_place
    except:
        print("Not found declaration_place")
    try: 
        spans = other_info_soup.find('table', id = "ctl00_cplhContainer_pnlCertificateOptions_radGridOptions_ctl00").find_all("span",class_ = "chkOption")
        remarks_info = []

        for span in spans:
            label = span.find("label").text.strip()  # Lấy nội dung của thẻ label
            input_tag = span.find("input")          # Lấy thẻ input bên trong
            checked = input_tag.has_attr("checked") # Kiểm tra trạng thái 'checked'
            remarks_info.append({"label": label, "checked": checked})
        id_info["remarks_info"] = remarks_info
    except:
        print("Not found remarks_info")
    print(id_info)
    return id_info

In [ ]:

# Hàm lưu trạng thái
def save_state(states, thread_index):
    file_path = state_file_template.format(thread_index)
    with open(file_path, "w") as f:
        json.dump(states, f, ensure_ascii=False, indent=4)  # Định dạng dễ đọc và tránh lỗi ghi

# Hàm đọc trạng thái
def load_state(thread_index):
    try:
        file_path = state_file_template.format(thread_index)
        if os.path.exists(file_path) and os.path.getsize(file_path) > 0:  # Kiểm tra file tồn tại và không trống
            with open(file_path, "r") as f:
                states = json.load(f)
                if isinstance(states, dict) and "start_index" in states:
                    return states
        # Nếu không hợp lệ hoặc không có 'start_index'
        return {"start_index": 0}
    except FileNotFoundError:
        return {"start_index": 0}
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {file_path}: {e}")
        return {"start_index": 0}  # Khởi tạo lại nếu lỗi


# Hàm xử lý từng chunk dữ liệu
def process_data_chunk(data_chunk, thread_index, start_index):
    all_id_info = [] 
    error_stt_list = []
    current_index = start_index

    while True:
        try:
            for index, item in enumerate(data_chunk[start_index:], start=start_index):
                try:
                    id_info = get_id_details(cookie, item)  # Hàm `get_id_details` cần được định nghĩa
                    all_id_info.append(id_info)

                    # Ghi kết quả vào file riêng cho thread
                    with open(output_file_template.format(thread_index), "a", encoding="utf-8") as f:
                        f.write(json.dumps(id_info, ensure_ascii=False) + "\n")
                except Exception as e:
                    print(f"Error processing row {index}: {e}")
                    error_stt_list.append({"stt": item, "error": str(e)})

                    # Ghi lỗi vào file
                    with open(error_file_template.format(thread_index), "a", encoding="utf-8") as f:
                        f.write(json.dumps({"stt": item, "error": str(e)}, ensure_ascii=False) + "\n")
                finally:
                    # Cập nhật trạng thái
                    current_index = index + 1
                    save_state({"start_index": current_index}, thread_index)
            return all_id_info, current_index, error_stt_list
        except Exception as e:
            print(f"Thread {thread_index} encountered an error: {e}. Restarting...")
            time.sleep(5)  # Đợi 5 giây trước khi thử lại

# Thư mục để lưu trạng thái và kết quả
output_dir = "org"
os.makedirs(output_dir, exist_ok=True)

state_file_template = os.path.join(output_dir, "loop_state_{}.json")
output_file_template = os.path.join(output_dir, "output_data_{}.txt")
error_file_template = os.path.join(output_dir, "error_stt_{}.txt")

# Chia danh sách thành các chunk
with open("output_co_ids.txt", "r", encoding="utf-8") as file:
    data_list = [line.strip() for line in file]

chunk_count = 5
chunk_size = -(-len(data_list) // chunk_count)
chunks = [data_list[i:i + chunk_size] for i in range(0, len(data_list), chunk_size)]

# Tải trạng thái đã lưu
saved_states = []
for i in range(len(chunks)):
    state = load_state(i)
    if "start_index" not in state:  # Đảm bảo có key 'start_index'
        state = {"start_index": 0}
    saved_states.append(state)

# Xử lý đa luồng
states = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=len(chunks)) as executor:
    futures = {
        executor.submit(process_data_chunk, chunk, idx, saved_states[idx]["start_index"]): idx
        for idx, chunk in enumerate(chunks)
    }
    for future in concurrent.futures.as_completed(futures):
        thread_index = futures[future]
        try:
            results, current_index, error_stts = future.result()
            states[thread_index] = {"start_index": current_index}
        except Exception as e:
            print(f"Thread {thread_index} encountered an error: {e}")
            save_state({"start_index": 0}, thread_index)

# Hợp nhất và lưu dữ liệu lỗi
error_file_names = [error_file_template.format(i) for i in range(chunk_count)]
all_error_data = []
for file_name in error_file_names:
    if os.path.exists(file_name):
        with open(file_name, "r", encoding="utf-8") as file:
            all_error_data.extend(json.loads(line) for line in file)

error_df = pd.DataFrame(all_error_data)
error_df.to_csv("all_error_co_id.csv", index=False, encoding="utf-8")

# Hợp nhất và lưu dữ liệu kết quả
output_file_names = [output_file_template.format(i) for i in range(chunk_count)]
all_output_data = []
for file_name in output_file_names:
    if os.path.exists(file_name):
        with open(file_name, "r", encoding="utf-8") as file:
            all_output_data.extend(json.loads(line) for line in file)

output_df = pd.DataFrame(all_output_data)
output_df.to_csv("all_co_id_info.csv", index=False, encoding="utf-8")


Request thành công!
{'doc_id': '11327132', 'Trạng thái hồ sơ': 'C/O đã được cấp phép\n(ĐÃ NỘP PHÍ)', 'Tax code': '1102022318', 'Issuing Authority': 'Phòng Quản lý XNK khu vực Tp. Hồ Chí Minh', 'Form': 'Form VK', 'Importing_Country': 'South Korea', 'Reference No.': 'VN-KR 23/02/027268', 'Issuance date': '02/11/2023', 'Certified date': '03/11/2023', 'ExportDeclaration': [{'item_num': 1, 'invoice_number': '305934242850', 'invoice_date': '30/10/2023', 'invoice_link': 'https://ecosys.gov.vn/Resources09/1102022318/2023/11/02/20231102094434_305934242850.pdf'}], 'Goods consigned from': ['ACE BONGSAN VINA COMPANY LIMITED', 'O2-10(02 FACTORY), ROAD 1B, KIZUNA 3 EXT, AREA K1, TAN KIM IP EXT', 'Can Giuoc Town, Can Giuoc District, Long An Province, Vietnam'], 'Goods consigned to': ['CHAI HONG CO.,LTD', '174, JAEJAEGI-RO, HWADO-EUP, NAMYANGJU-SI,', 'GYEONGGI-DO, (ZIP12200) KOREA'], 'transport_type': 'By Sea', 'Departure Date': '01/11/2023', 'vessel_aircraft_name': 'WAN HAI 283 N062', 'port_of_loadin